In [1]:
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk
import plotly_express as px
import configparser
import os
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk
import plotly_express as px
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
from numpy.linalg import norm
from scipy.spatial.distance import pdist
import plotly_express as px
import seaborn as sns; sns.set()
import configparser
config = configparser.ConfigParser()
config.read("../../../env-sample.ini")
data_home = '/Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/data'
output_dir = '/Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/output'
local_lib = '/Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/DS5001-2025-01-R/lessons/lib'
import sys
sys.path.append(local_lib)
from textparser import TextParser

clip_pats = [
    r"\*\*\*\s*START OF",
    r"\*\*\*\s*END OF"
]

# All are 'chap'and 'm'
roman = '[IVXLCM]+'
caps = "[A-Z';, -]+"
ohco_pat_list = [
    (5200,   rf"^\s*CHAPTER\s+{roman}\s*$"), #Metamorphosis
    (7849,   rf"^\s*{roman}\s*$"), #The Trial
    (6969,  rf"^\s*LETTER .* to .*$"), # The Castle
    (6262,   rf"^CHAPTER\s+{roman}$"), # Amerika
    (6161,   rf"^CHAPTER\s+\d+$"), # The Judgement
    (6060,   rf"^Chapter\s+\d+$"), # Dearest Father
    (6363,  rf"^Chapter\s+\d+$"), # In the Penal colony
    (6464,   rf"^CHAPTER\s+\d+$"), # The Hunger Artist
    (6565, rf"^\s*CHAPTER\s+{roman}\."), # The Jackals and Arabs
    (6666, rf"^\s*CHAPTER\s+{roman}\s*$"), # A Country Doctor
    (6767, rf"^\s*CHAPTER\s+{roman}\s*$"), # An Imperial Message
    (5959,  rf"^(?:ETYMOLOGY|EXTRACTS|CHAPTER)"), # A report for an Academy
    (5858,  rf"^\s*CHAPTER\s+{roman}\.\s*$"), # The Great Wall of China
    (5757, rf"^\s*{roman}\.\s*$"), # The Hunter Gracchus
    (5656,  rf"^\s*{roman}\. .*$"), # Up in the Gallery
    (5555, rf"^CHAPTER\s+{roman}\.?$"), # Before the Law
    (5454, rf"^\s*[A-Z,;-]+\.\s*$"), # Josephine the Songstress
    (5353,  rf"^CHAPTER "), # The Burrow
    (5252, rf"^CHAPTER\s+{roman}\.\s*$"), # Blumfeld
    (23532, rf"Chapter\s+{roman}") # Meditation
]
chapter_regexes = [
    (5200,   rf"^\s*{roman}\s*$"),
    (7849,   rf"^\s*Chapter\s+(?:One|Two|Three|Four|Five|Six|Seven|Eight|Nine|Ten)\s*$"),
    (6969,   rf"^\s*\d+\s*$"),
    (6262,   rf"^\s*\d+\s*$"),
    (6161,   "NOCHAPTERS"),
    (6060,   "NOCHAPTERS"),
    (6363,   "NOCHAPTERS"),
    (6464,   "NOCHAPTERS"),
    (6565,   "NOCHAPTERS"),
    (6666,   "NOCHAPTERS"),
    (6767,   "NOCHAPTERS"),
    (5959,   "NOCHAPTERS"),
    (5858,   "NOCHAPTERS"),
    (5757,   "NOCHAPTERS"),
    (5656,   "NOCHAPTERS"),
    (5555,   "NOCHAPTERS"),
    (5454,   "NOCHAPTERS"),
    (5353,   "NOCHAPTERS"),
    (5252,   "NOCHAPTERS"),
    (23532,  rf"^(Children on the country road|Unmasking a con artist|The Sudden Walk|Resolutions|The trip to the mountains|The Bachelor's Misfortune|The Merchant|Distracted Looking Out|The Way Home|The Passers-by|Passenger|Dresses|The rejection|Food for thought for gentlemen riders|The Alley Window|Desire to become an Indian|The Trees|Unhappiness)$")  # Poem title on line 1
]
ohco_pat_list = chapter_regexes
source_files = f'{data_home}'
source_file_list = sorted(glob(f"{source_files}/*.*"))

book_data = []
for source_file_path in source_file_list:
    # Get the filename only, e.g. 'pg5353.txt'
    filename = os.path.basename(source_file_path)
    # Extract the numeric ID from the filename (remove 'pg' and '.txt')
    book_id = int(filename.replace('pg', '').replace('.txt', ''))
    # Use filename (without extension) as a raw title (optional: clean further)
    book_title = filename.replace('.txt', '').replace('_', ' ')
    # Append a tuple of (book_id, path, title)
    book_data.append((book_id, source_file_path, book_title))
# Convert to DataFrame
LIB = pd.DataFrame(book_data, columns=['book_id', 'source_file_path', 'raw_title']) \
        .set_index('book_id') \
        .sort_index()
book_titles = {
    5200: "Metamorphosis",
    7849: "The Trial",
    6969: "The Castle",
    6262: "Amerika",
    6161: "The Judgement",
    6060: "Dearest Father",
    6363: "In the Penal Colony",
    6464: "The Hunger Artist",
    6565: "The Jackals and Arabs",
    6666: "A Country Doctor",
    6767: "An Imperial Message",
    5959: "A Report for an Academy",
    5858: "The Great Wall of China",
    5757: "The Hunter Gracchus",
    5656: "Up in the Gallery",
    5555: "Before the Law",
    5454: "Josephine the Songstress",
    5353: "The Burrow",
    5252: "Blumfeld",
    23532: "Meditation"
}
book_titles = {f'pg{key}': value for key, value in book_titles.items()}
try:
    LIB['author'] = 'KAFKA, FRANZ'
    LIB['title'] = LIB.raw_title.replace(book_titles).str.upper()
    LIB = LIB.drop('raw_title', axis=1)
except AttributeError:
    pass
LIB['chap_regex'] = LIB.index.map(pd.Series({x[0]:x[1] for x in ohco_pat_list}))
LIB
# This cell takes 16 seconds to run
def tokenize_collection(LIB):

    clip_pats = [
        r"\*\*\*\s*START OF",
        r"\*\*\*\s*END OF"
    ]

    books = []
    for book_id in LIB.index:

        # Announce
        print("Tokenizing", book_id, LIB.loc[book_id].title)

        # Define vars
        chap_regex = LIB.loc[book_id].chap_regex
        ohco_pats = [('chap', chap_regex, 'm')]
        src_file_path = LIB.loc[book_id].source_file_path

        # Create object
        text = TextParser(src_file_path, ohco_pats=ohco_pats, clip_pats=clip_pats, use_nltk=True)
        # text = TextImporter(src_file_path, ohco_pats=ohco_pats, clip_pats=clip_pats) 

        # Define parameters
        text.verbose = True
        text.strip_hyphens = True
        text.strip_whitespace = True

        # Parse
        text.import_source().parse_tokens();

        # Name things
        text.TOKENS['book_id'] = book_id
        text.TOKENS = text.TOKENS.reset_index().set_index(['book_id'] + text.OHCO)

        # Add to list
        books.append(text.TOKENS)
        
    # Combine into a single dataframe
    CORPUS = pd.concat(books).sort_index()

    # Clean up
    del(books)
    del(text)
        
    print("Done")
        
    return CORPUS
CORPUS = tokenize_collection(LIB)


Tokenizing 5200 METAMORPHOSIS
Importing  /Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/data\pg5200.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^\s*[IVXLCM]+\s*$
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model
Parsing OHCO level 3 token_num by NLTK model
Tokenizing 5252 BLUMFELD
Importing  /Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/data\pg5252.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone NOCHAPTERS
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model
Parsing OHCO level 3 token_num by NLTK model
Tokenizing 5353 THE BURROW
Importing  /Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/data\pg5353.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone NOCHAPTERS
li

c:\Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/DS5001-2025-01-R/lessons/lib\textparser.py:132: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  div_lines = self.TOKENS[src_col].str.contains(div_pat, regex=True, case=True)


Done


In [2]:
CORPUS = CORPUS[CORPUS.term_str != '']
CORPUS['pos_group'] = CORPUS.pos.str[:2]
CORPUS

pos_tuple  pos token_str  \
book_id chap_id para_num sent_num token_num                                  
5200    1       1        0        0               (One, CD)   CD       One   
                                  1          (morning,, NN)   NN  morning,   
                                  2             (when, WRB)  WRB      when   
                                  3           (Gregor, NNP)  NNP    Gregor   
                                  4            (Samsa, NNP)  NNP     Samsa   
...                                                     ...  ...       ...   
23532   10      39       0        27         (upstairs, JJ)   JJ  upstairs   
                                  28              (and, CC)   CC       and   
                                  29               (go, VB)   VB        go   
                                  30               (to, TO)   TO        to   
                                  31           (sleep., VB)   VB    sleep.   

                                             term_str pos_group  
book_id chap_id para_num sent_num token_num                      
5200    1       1        0        0               one        CD  
                                  1           morning        NN  
                                  2              when        WR  
                                  3            gregor        NN  
                                  4             samsa        NN  
...                                               ...       ...  
23532   10      39       0        27         upstairs        JJ  
                                  28              and        CC  
                                  29               go        VB  
                                  30               to        TO  
                                  31            sleep        VB  

[423144 rows x 5 columns]

In [3]:
VOCAB = CORPUS.term_str.value_counts().to_frame('n').sort_index()
VOCAB.index.name = 'term_str'
VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)
VOCAB['max_pos'] = CORPUS[['term_str','pos']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['max_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['n_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().unstack().count(1)
VOCAB['cat_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().to_frame('n').reset_index()\
    .groupby('term_str').pos_group.apply(lambda x: set(x))
VOCAB['n_pos'] = CORPUS[['term_str','pos']].value_counts().unstack().count(1)
VOCAB['cat_pos'] = CORPUS[['term_str','pos']].value_counts().to_frame('n').reset_index()\
    .groupby('term_str').pos.apply(lambda x: set(x))
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1
VOCAB['stop'] = VOCAB.index.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')
from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['porter_stem'] = VOCAB.apply(lambda x: stemmer1.stem(x.name), 1)

from nltk.stem.snowball import SnowballStemmer
stemmer2 = SnowballStemmer("english")
VOCAB['stem_snowball'] = VOCAB.apply(lambda x: stemmer2.stem(x.name), 1)

from nltk.stem.lancaster import LancasterStemmer
stemmer3 = LancasterStemmer()
VOCAB['stem_lancaster'] = VOCAB.apply(lambda x: stemmer3.stem(x.name), 1)

VOCAB

,n,n_chars,p,i,max_pos,max_pos_group,n_pos_group,cat_pos_group,n_pos,cat_pos,stop,porter_stem,stem_snowball,stem_lancaster
term_str,,,,,,,,,,,,,,
1,1,1,0.000002,18.690615,JJ,JJ,1,{JJ},1,{JJ},0,1,1,1
10,1,2,0.000002,18.690615,CD,CD,1,{CD},1,{CD},0,10,10,10
15,1,2,0.000002,18.690615,CD,CD,1,{CD},1,{CD},0,15,15,15
2,1,1,0.000002,18.690615,NN,NN,1,{NN},1,{NN},0,2,2,2
25,1,2,0.000002,18.690615,CD,CD,1,{CD},1,{CD},0,25,25,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ﬂung,6,4,0.000014,16.105653,NNP,NN,4,"{VB, IN, JJ, NN}",5,"{NN, IN, JJ, VBP, NNP}",0,ﬂung,ﬂung,ﬂung
ﬂushed,1,6,0.000002,18.690615,NN,NN,1,{NN},1,{NN},0,ﬂush,ﬂush,ﬂush
ﬂuttered,1,8,0.000002,18.690615,VBN,VB,1,{VB},1,{VBN},0,ﬂutter,ﬂutter,ﬂut


In [5]:
VOCAB.columns.to_list()

['n',
 'n_chars',
 'p',
 'i',
 'max_pos',
 'max_pos_group',
 'n_pos_group',
 'cat_pos_group',
 'n_pos',
 'cat_pos',
 'stop',
 'porter_stem',
 'stem_snowball',
 'stem_lancaster']

In [6]:
# Functions to create TFIDF:
# Bring into your notebook the functions you created previously to generate a BOW table and compute TFIDF values. 
# Extend the TFIDF function so that it also returns the DFIDF value for each term in the VOCAB.
def gather_docs(CORPUS, ohco_level, term_col='term_str'):
    OHCO = CORPUS.index.names
    CORPUS[term_col] = CORPUS[term_col].astype('str')
    DOC = CORPUS.groupby(OHCO[:ohco_level])[term_col].apply(lambda x:' '.join(x)).to_frame('doc_str')
    return DOC

def BOW(tokendf, ocholevel):
    return tokendf.groupby(bags[ocholevel]+['term_str']).term_str.count().to_frame('n') 

def BOWtoTFIDF(BOW, tf_method, CORPUS, ocholevel = 2):
    
    # I added another parameter for the CORPUS because otherwise it becomes really tedious to undo the BOW function

    DTCM = BOW.n.unstack(fill_value=0)
    if tf_method == 'sum':
        TF = DTCM.T / DTCM.T.sum()
    elif tf_method == 'max':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'log':
        TF = np.log2(1 + DTCM.T)
    elif tf_method == 'raw':
        TF = DTCM.T
    elif tf_method == 'double_norm':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
    TF = TF.T
    DF = DTCM.astype('bool').sum() 
    N = DTCM.shape[0]
    IDF = IDF = np.log2(N / DF)
    TFIDF = TF * IDF
    # Extend the TFIDF function so that it also returns the DFIDF value for each term in the VOCAB.

    # We can adjust ocho level here if we want to, default is 2
    DOC = gather_docs(CORPUS, 2)
    DOC['n_tokens'] = DOC.doc_str.apply(lambda x: len(x.split()))
    ngram_range = (1,2)
    n_terms = 4000
    count_engine = CountVectorizer(
        stop_words = 'english',
        ngram_range = ngram_range,
        max_features = n_terms)
    X = count_engine.fit_transform(DOC.doc_str)
    DTM = pd.DataFrame(X.toarray(), 
    columns=count_engine.get_feature_names_out(), 
    index=DOC.index)
    VOCAB = DTM.sum().to_frame('n')
    VOCAB.index.name = 'term_str'
    VOCAB['df'] = DTM.astype(bool).sum()
    VOCAB['dfidf'] = VOCAB.df * np.log2(len(DTM)/VOCAB.df)
    VOCAB['dp'] = VOCAB.df / len(DTM)
    VOCAB['di'] = np.log2(1/VOCAB.dp)
    VOCAB['dh'] = VOCAB.dp * VOCAB.di
    VOCAB['n_chars'] = VOCAB.apply(lambda x: len(x.name), 1)
    VOCAB['n_tokens'] = VOCAB.apply(lambda x: len(x.name.split()), 1)
    VOCAB.sort_index()


    #Hopefully this join works but I'm actually not too sure this will work
    
    return TFIDF,VOCAB,DTM

# I think I'm going to use this oneliner more.
# get_tfidf = lambda X, agg_func='sum': (X.T / X.T.agg(agg_func)).T * (np.log2(len(X)/X.astype('bool').sum()))
OHCO = ['book_id','chap_id', 'para_num', 'sent_num', 'token_num']

bags = dict(
    SENTS = OHCO[:4],
    PARAS = OHCO[:3],
    CHAPS = OHCO[:2],
    BOOKS = OHCO[:1]
)
tables = BOWtoTFIDF(BOW(CORPUS,'CHAPS'),'max', CORPUS, 2)
DTM = tables[2]
DFIDF = tables[1]
TFIDF = tables[0]

In [9]:
DFIDF['dfidf']

term_str
abandoned     34.710943
abandoning    21.265923
abilities     23.253497
ability       28.073549
able          18.027973
                ...    
ﬁrst time     25.034264
ﬁst           23.253497
ﬁt            25.034264
ﬁxed          21.265923
ﬂoor          30.531846
Name: dfidf, Length: 4000, dtype: float64

In [13]:
VOCAB = VOCAB.join(DFIDF['dfidf'])
VOCAB['dfidf'] = VOCAB['dfidf'].fillna(0)

In [17]:
VOCAB.sort_values('dfidf',ascending=False).head(20)

,n,n_chars,p,i,max_pos,max_pos_group,n_pos_group,cat_pos_group,n_pos,cat_pos,stop,porter_stem,stem_snowball,stem_lancaster,dfidf
term_str,,,,,,,,,,,,,,,
reasons,50,7,0.000118,13.046759,NNS,NN,2,"{VB, NN}",3,"{VB, NNS, NN}",0,reason,reason,reason,37.149926
rooms,81,5,0.000191,12.350765,NNS,NN,3,"{VB, JJ, NN}",4,"{VBZ, NNS, JJ, NN}",0,room,room,room,37.149926
white,54,5,0.000128,12.935728,JJ,JJ,2,"{JJ, NN}",2,"{JJ, NN}",0,white,white,whit,37.149926
fetch,52,5,0.000123,12.990176,VB,VB,2,"{VB, NN}",3,"{VB, VBP, NN}",0,fetch,fetch,fetch,37.149926
appearance,42,10,0.000099,13.298298,NN,NN,2,"{JJ, NN}",2,"{JJ, NN}",0,appear,appear,appear,37.149926
changed,50,7,0.000118,13.046759,VBN,VB,3,"{VB, JJ, NN}",7,"{VBZ, NN, JJ, VBN, VBP, NNS, VBD}",0,chang,chang,chang,37.149926
calmly,41,6,0.000097,13.333063,RB,NN,4,"{VB, JJ, RB, NN}",8,"{VB, VBZ, NN, JJ, VBP, NNS, VBD, RB}",0,calmli,calm,calm,37.149926
thrown,45,6,0.000106,13.198762,VBN,VB,5,"{VB, NN, IN, JJ, RP}",7,"{VB, NN, IN, JJ, VBN, VBP, RP}",0,thrown,thrown,thrown,37.149926
walk,66,4,0.000156,12.646221,VB,VB,2,"{VB, NN}",3,"{VB, VBP, NN}",0,walk,walk,walk,37.149926


In [20]:
#VOCAB.to_csv('VOCAB.csv')